In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import cPickle as pickle
import codecs

from operator import itemgetter
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster.bicluster import SpectralCoclustering
from biclustering.biclustering import DeltaBiclustering
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

In [2]:
%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})

In [3]:
arena_news_stem_df = pd.read_pickle('arena_news_stem_df.pkl')
sport_news_stem_df = pd.read_pickle('sport_news_stem_df.pkl')
jovem_news_stem_df = pd.read_pickle('jovem_news_stem_df.pkl')
arena_news_df = pd.read_pickle('arena_news_df.pkl')
sport_news_df = pd.read_pickle('sport_news_df.pkl')
jovem_news_df = pd.read_pickle('jovem_news_df.pkl')

In [4]:
labels = np.array(len(arena_news_df)*[1] + len(sport_news_df)*[0])

In [5]:
all_news_df = arena_news_df.append(sport_news_df)
all_news_df.reset_index(inplace=True)
all_news_df.drop('index', axis=1, inplace=True)
all_news_df

,url,title,description,body,all
0,http://arena.ig.com.br/2014-10-09/os-campeonat...,Os campeonatos da 'Brasil Game Show',Confira os campeonatos que rolam no evento,"O ""Brasil Game Show"" acontece de 09 até 12/10...",campeonatos brasil game show confira campeonat...
1,http://arena.ig.com.br/2014-09-09/conheca-o-jo...,Conheça o jogo usado para lançar o iPhone 6,"Vainglory é um multiplayer de batalha, no esti...",\n\t \n\t \t\t\t\t\n\t\t Divulgação \n\t\t Des...,conheça jogo usado lançar iphone vainglory mul...
2,http://arena.ig.com.br/2014-09-25/avaliacao-do...,Avaliação do FIFA 15 por um jogador fanático,Convidamos um jogador maníaco por FIFA para di...,Aspectos Positivos Estádios A novidade é que o...,avaliação fifa jogador fanático convidamos jog...
3,http://arena.ig.com.br/2014-09-11/paul-mccartn...,Paul McCartney escreveu música para Destiny se...,Novo single “Hope for the Future” toca ao fina...,\n\t \n\t \t\t\t\t\n\t\t Reprodução \n\t\t Pau...,paul mccartney escreveu música destiny interes...
4,http://arena.ig.com.br/especiais/infograficos/...,Infográfico: A evolução dos controles em 40 an...,Relembramos e detalhamos (quase) todos os cont...,"É possível que, ao segurar um controle do Odys...",evolução controles anos história videogames re...
5,http://arena.ig.com.br/2012-11-29/a-industria-...,A indústria fantasma em busca do Peixonauta do...,Com a participação de representantes de difere...,\n \n \...,indústria fantasma busca peixonauta games part...
6,http://arena.ig.com.br/2012-07-24/festival-big...,Festival BIG trará co-criador de Halo a SP; ve...,"Evento poderá ter reality show, fase de Angry ...",\n \n \...,festival big trará co criador halo sp novidade...
7,http://arena.ig.com.br/2014-01-17/um-papo-com-...,"Um papo com Amora: TowerFall, MiniBoss e uma b...","Em entrevista ao Arena, artista do MiniBoss co...",Faz apenas algumas semanas desde que Raquel O...,papo amora towerfall miniboss bagagem cheia no...
8,http://arena.ig.com.br/2013-05-24/aprovado-no-...,"Aprovado no Greenlight, Contrast chega para PC...","Puzzle 2D-3D tem atmosfera vintage, com cabare...",\n\t \n\t \t\t\t\t\n\t\t Reprodução \n\t\t Con...,aprovado greenlight contrast chega pc xbla psn...
9,http://arena.ig.com.br/2013-02-21/jogos-da-psn...,Jogos da PSN comprados no PS3 não poderão ser ...,"Retrocompatibilidade por nuvem, porém, permiti...",\n\t \n\t \t\t\t\t\n\t\t Reprodução \n\t\t Pla...,jogos psn comprados ps poderão transferidos ps...


In [17]:
for news in arena_news_df['all']:
    c = 0
    if u'playstation' in news or u'Playstation' in news or u'PlayStation' in news:
        c += 1
        print news
print c

papo amora towerfall miniboss bagagem cheia novas experiências entrevista arena artista miniboss conta trabalhar jogos aclamados aprendeu vivenciou durante três meses canadá faz apenas semanas desde raquel oliveira pedro medeiros retornaram paulo três meses trabalho intenso dupla novas descobertas amadurecimento fernando piovesan raquel oliveira conhecida nome artístico amora bettany durante período conviveram cinco pessoas incluindo alec holowka responsáveis aquaria matt thorson criador towerfall casa vancouver canadá indie house intitulada moradores compartilharam trabalho towerfall finalmente deixa ouya chegar plataformas pc playstation fevereiro towerfall ascension raquel conhecida amora pedro santo ambos compõem miniboss selo jogos independentes criado dupla anos destaca cena indie brasil towerfall towerfall eleito melhor jogo melhor jogo multiplayer ano arena brasil marca presença premiação igf towerfall ascension amora amigo longa data rafa miranda responsável músicas jogos mini

In [19]:
count_vect = CountVectorizer(encoding='UTF-8',lowercase=False,min_df=2)
X = count_vect.fit_transform(arena_news_df['all'].tolist() + sport_news_df['all'].tolist())

X_train_norm_tfidf = TfidfTransformer(norm=u'l2', use_idf=True).fit_transform(X)
X_train_norm = TfidfTransformer(norm=u'l2', use_idf=False).fit_transform(X)

In [49]:
def to_csv(some_list, file_name, header):
    def to_str(st):
        if isinstance(st, str) or isinstance(st, unicode):
            return st
        else:
            return str(st)
    with codecs.open(file_name, 'w', 'utf-8') as file_stream:
        file_stream.write(u','.join(header) + '\n')
        for item in some_list:
            if isinstance(item, tuple) or isinstance(item, list):
                line = u','.join(map(to_str, item)) + '\n'
            else:
                line = item + '\n'
            file_stream.write(line)
    file_stream.close()

# All words top 20

In [59]:
def top_k(arr, k, axis=0):
    top_inds = np.argsort(arr, axis=axis)[-k:]
    top_vals = np.sort(arr, axis=axis)[-k:]
    return top_inds, top_vals

def reverse(arr):
    return arr[::-1]

def top_k_words(vec, count_vect, k):
    top_inds, top_vals = top_k(vec, k)
    words = count_vect.get_feature_names()
    top_words = [words[i] for i in top_inds]

    top_pairs = reverse(zip(top_words, top_vals))

    return top_pairs

sum_per_word = np.sum(X_train_norm.toarray(), axis=0)
top_pairs = top_k_words(sum_per_word, count_vect, 9999999)
for word, value in top_pairs[0:20]:
    print '%s, %s' % (word, value)
to_csv(top_pairs, 'all_words_top.csv', ['word', 'tf norm value'])

jogo, 17.409134665
jogos, 14.0184074837
of, 6.66858864083
feira, 6.63423662207
playstation, 6.50286252979
novo, 6.4397883303
xbox, 5.93517551309
games, 5.41506727974
game, 5.40202523592
equipe, 5.38402375813
dia, 5.32093077415
time, 5.21085736446
final, 4.9373205725
anos, 4.91571457583
brasileiro, 4.87632646275
monetáriointerno, 4.81840908599
brasil, 4.75295749281
ano, 4.75079310834
jogador, 4.68028023127
segundo, 4.64121947369


# Arena words top 20

In [60]:
sum_per_word = np.sum(X_train_norm.toarray()[0:100, :], axis=0)
top_pairs = top_k_words(sum_per_word, count_vect, 9999999)
for word, value in top_pairs[0:20]:
    print '%s, %s' % (word, value)
to_csv(top_pairs, 'arena_words_top.csv', ['word', 'tf norm value'])

jogo, 13.0195645784
jogos, 12.3527084592
of, 6.59321080468
playstation, 6.50286252979
xbox, 5.93517551309
games, 5.41506727974
game, 5.29370574087
novo, 4.93748565046
console, 4.56307259533
sony, 4.15619803355
the, 3.97710024088
wii, 3.89686450689
brasil, 3.6993005896
nintendo, 3.27248181662
legends, 3.23393553266
league, 3.20646821137
apenas, 2.82897655817
one, 2.70936450188
dia, 2.69523330746
lançado, 2.55713526763


# Sports words top 20

In [1]:
sum_per_word = np.sum(X_train_norm[100:200, :], axis=0)
top_pairs = top_k_words(sum_per_word, count_vect, 9999999)
for word, value in top_pairs[0:20]:
    print '%s, %s' % (word, value)
to_csv(top_pairs, 'sports_words_top.csv', ['word', 'tf norm value'])

NameError: name 'np' is not defined